In [ ]:
import os
import cdsapi
import zipfile
import numpy as np
import pandas as pd
import geopandas as gpd
from netCDF4 import Dataset
from scipy.interpolate import NearestNDInterpolator, griddata

METHOD = 'cubic' # near linear cubic

completed = []
with open('климат добавлен.txt', 'r') as f:
    completed = [s.replace('\n', '') for s in f.readlines()]

c = cdsapi.Client()

dirname = "/home/prokofev.a@agtu.ru/Загрузки/qgis temp/точки/сокращено/"

if not os.path.isdir('./климатические данные/'):
    os.mkdir('./климатические данные/')
if not os.path.isdir('./климатические данные/распакованные архивы/'):
    os.mkdir('./климатические данные/распакованные архивы/')

dataNames = {
    #"data_stream-wave_stepType-instant.nc": {
    #    "mwd": "mean_wave_direction"
    #},
    "data_stream-oper_stepType-instant.nc": {
        "u10": "10m_u_component_of_wind",
        "v10": "10m_v_component_of_wind",
        "t2m": "2m_temperature",
        "sst": "sea_surface_temperature",
        "sp": "surface_pressure",
        "rsn": "snow_density",
        "sd": "snow_depth",
        "lsm": "land_sea_mask"
    },
    "data_stream-oper_stepType-accum.nc": {
        "ssrd": "surface_solar_radiation_downwards",
        "strd": "surface_thermal_radiation_downwards",
        "e": "evaporation",
    }
}

for fname in os.listdir(dirname):
    if fname.endswith(".gpkg"):
        if fname not in completed:
            data = gpd.read_file(f"{dirname}{fname}").sort_values(by=['point_id'])

            ymax = data.geometry.y.max()
            ymin = data.geometry.y.min()
            xmax = data.geometry.x.max()
            xmin = data.geometry.x.min()

            year = fname.split("_", 5)[-2]
            year, hour = year.split("T")
            year, month, day = year[0:4], year[4:6], year[6:]
            hour, minute, second = int(hour[0:2]), int(hour[2:4]), int(hour[4:])
            minute = minute + 1 if int(second) >= 30 else minute
            hour = "{:02d}".format(hour if minute < 30 else hour + 1)
            year, month, day, hour, minute, second

            if not os.path.isfile('./климатические данные/' + fname.replace(".gpkg", ".nc")):
                try:
                    c.retrieve(
                        'reanalysis-era5-single-levels',
                        {
                            'variable': [v for j in dataNames.values() for v in j.values()],
                            'product_type': ['reanalysis'],
                            'year': [year],
                            'month': [month],
                            'day': [day],
                            'time': [f'{hour}:00'],
                            'data_format': 'netcdf',
                            'download_format': 'unarchived',
                            'area': [ymax + 0.1, xmin - 0.1, ymin - 0.1, xmax + 0.1],
                            'grid': [round(float((xmax - xmin) / 10), 3), round(float((ymax - ymin) / 10), 3)]
                        },
                        './климатические данные/' + fname.replace(".gpkg", ".nc")
                    )
                except:
                    continue
            
            with zipfile.ZipFile('./климатические данные/' + fname.replace(".gpkg", ".nc"), 'r') as zip_file:
                zip_file.extractall('./климатические данные/распакованные архивы/')

            ncdata = pd.DataFrame([])

            for nc in os.listdir('./климатические данные/распакованные архивы/'):
                dset = Dataset(f'./климатические данные/распакованные архивы/{nc}', 'r')
                for key in dataNames[nc].keys():
                    ncdata[key] = dset.variables[key][:].flatten().data
                if nc == list(dataNames.keys())[0]:
                    nc_lat = dset.variables['latitude'][:].flatten().data
                    nc_lon = dset.variables['longitude'][:].flatten().data
                    ncdata['latitude'] = np.stack([np.full(shape=len(nc_lon), fill_value=i, dtype=np.float32) for i in nc_lat]).flatten()
                    ncdata['longitude'] = np.stack([nc_lon] * len(nc_lat)).flatten()
                os.remove(f'./климатические данные/распакованные архивы/{nc}')

            if METHOD == 'near':
                points = np.array(list(ncdata[['longitude', "latitude"]].itertuples(index=False, name=None)))
                geometry = data.geometry.get_coordinates()[['x', 'y']]

                for col in ncdata.columns:
                    if col not in ("latitude",'longitude'):
                        interp = NearestNDInterpolator(points, ncdata[col])
                        data[col] = interp(geometry)
            else:
                points = np.array(list(ncdata[['longitude', "latitude"]].itertuples(index=False, name=None)))
                geometry = (data['geometry'].x, data['geometry'].y)

                for col in ncdata.columns:
                    if col not in ("latitude",'longitude'):
                        data[col] = griddata(points, ncdata[col], geometry, method=METHOD)
            data.to_file('./климатические данные/' + fname)

            with open('климат добавлен.txt', 'a') as f:
                f.write(fname + '\n')

2025-04-08 13:40:08,642 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-08 13:40:08,645 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-04-08 13:40:11,299 INFO Request ID is b004fbd3-fb4d-499a-a7af-99951ee4f9ba
2025-04-08 13:40:11,392 INFO status has been updated to accepted
2025-04-08 13:40:44,308 INFO status has been updated to running
2025-04-08 13:41:01,486 INFO status has been updated to successful
2025-04-08 13:41:11,599 INFO Request ID is 7bbd79c4-4cf5-485a-98d8-ebbbe67cab93        
2025-04-08 13:41:11,697 INFO status has been updated to accepted
2025-04-08 13:41:16,776 INFO status has been updated to running
2025-04-08 13:41:20,237 INFO status has been updated to succ

In [ ]:
data

In [ ]:
ncdata['geometry'] = gpd.points_from_xy(ncdata['longitude'], ncdata['latitude'])
gpd.GeoDataFrame(ncdata, crs='EPSG:4326').to_file('nc.gpkg')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

image_data = np.reshape(data["t2m"].to_numpy(dtype=np.int32), (1000, 1000), order="F")
plt.imshow(image_data, cmap='gray')
plt.show()